<a href="https://colab.research.google.com/github/Hanbin-git/kaggle/blob/main/RNA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

stanford_rna_3d_folding_path = kagglehub.competition_download('stanford-rna-3d-folding')
metric_usalign_path = kagglehub.dataset_download('metric/usalign')
geraseva_protenix_checkpoints_path = kagglehub.dataset_download('geraseva/protenix-checkpoints')
metric_ribonanza_tm_score_path = kagglehub.notebook_output_download('metric/ribonanza-tm-score')

print('Data source import complete.')


In [ ]:
MODEL_TYPE='protenix'
VALIDATION=True

In [ ]:
!pip uninstall -y torch torchvision torchaudio protenix

Found existing installation: torch 2.1.2+cu118
Uninstalling torch-2.1.2+cu118:
  Successfully uninstalled torch-2.1.2+cu118
Found existing installation: protenix 0.4.6
Uninstalling protenix-0.4.6:
  Successfully uninstalled protenix-0.4.6


In [ ]:
!pip install --extra-index-url https://download.pytorch.org/whl/cu118 torch==2.1.2+cu118

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.1.2%2Bcu118-cp310-cp310-linux_x86_64.whl (2325.9 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
easyocr 1.7.2 requires torchvision>=0.5, which is not installed.
fastai 2.7.18 requires torchvision>=0.11, which is not installed.
protenix 0.4.6 requires deepspeed>=0.15.1, which is not installed.
protenix 0.4.6 requires icecream, which is not installed.
protenix 0.4.6 requires ipdb, which is not installed.
protenix 0.4.6 requires modelcif==0.7, which is not installed.
protenix 0.4.6 requires py3Dmol, which is not installed.
protenix 0.4.6 requires scikit-learn-extra, which is not installed.
timm 1.0.12 requires torchvision, which is not installed.
protenix 0.4.6 requires biopython==1.83, but you have biopython 1.85 w

In [ ]:
!pip install --no-deps protenix==0.4.6 einops tqdm gemmi biotite==1.0.1

  Using cached protenix-0.4.6-py3-none-any.whl.metadata (1.1 kB)
Using cached protenix-0.4.6-py3-none-any.whl (441 kB)


## Install requirements

In [ ]:
if MODEL_TYPE=='protenix' and VALIDATION:
    !pip install biopython
    !pip install ml-collections
    !pip install rdkit
!export PROTENIX_DATA_ROOT_DIR=/kaggle/input/protenix-checkpoints

In [ ]:
! mkdir /af3-dev
! ln -s /kaggle/input/protenix-checkpoints /af3-dev/release_data
! ls /af3-dev/release_data/

mkdir: cannot create directory ‘/af3-dev’: File exists
ln: failed to create symbolic link '/af3-dev/release_data/protenix-checkpoints': Read-only file system
components.v20240608.cif		model_v0.2.0.pt
components.v20240608.cif.rdkit_mol.pkl


## Helper scripts

In [ ]:
import Bio

from copy import deepcopy

import pandas as pd
from Bio.PDB import Atom, Model, Chain, Residue, Structure, PDBParser
from Bio import SeqIO
import os, sys
import re
import numpy as np
import torch

import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
time0=time.time()

print('IMPORT OK !!!!')

IMPORT OK !!!!


In [ ]:
import os, sys, re
import numpy as np
import pandas as pd
from Bio.PDB import PDBParser

# 🧱 환경 설정
PYTHON = sys.executable
print('PYTHON', PYTHON)

PYTHON /usr/bin/python3


In [ ]:
RHONET_DIR = '/kaggle/input/data-for-demo-for-rhofold-plus-with-kaggle-msa/RhoFold-main'
USALIGN = '/kaggle/working/USalign'

os.system('cp /kaggle/input/usalign/USalign /kaggle/working/')
os.system('chmod u+x /kaggle/working/USalign')  # sudo는 필요 없음
sys.path.append(RHONET_DIR)

DATA_KAGGLE_DIR = '/kaggle/input/stanford-rna-3d-folding'

# 🧾 dotdict
class dotdict(dict):
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

    def __getattr__(self, name):
        try:
            return self[name]
        except KeyError:
            raise AttributeError(name)

# 📊 시각화용
def set_aspect_equal(ax):
    x_limits = ax.get_xlim()
    y_limits = ax.get_ylim()
    z_limits = ax.get_zlim()
    x_middle, y_middle, z_middle = np.mean(x_limits), np.mean(y_limits), np.mean(z_limits)
    max_range = max(
        x_limits[1] - x_limits[0],
        y_limits[1] - y_limits[0],
        z_limits[1] - z_limits[0]
    ) / 2.0
    ax.set_xlim(x_middle - max_range, x_middle + max_range)
    ax.set_ylim(y_middle - max_range, y_middle + max_range)
    ax.set_zlim(z_middle - max_range, z_middle + max_range)

# 📌 예측 output → DataFrame 변환
def parse_output_to_df(output, seq, target_id):
    if isinstance(output, torch.Tensor):
        output = output.detach().cpu().numpy()
    output = output.squeeze()  # [L, 3]
    chain_data = []
    for i, res in enumerate(seq):
        x, y, z = output[i]
        chain_data.append(dict(
            ID=target_id,
            resname=res,
            resid=i + 1,
            x_1=round(x, 3),
            y_1=round(y, 3),
            z_1=round(z, 3)
        ))
    return [pd.DataFrame(chain_data)]

# 📌 GT PDB → DataFrame 변환
def parse_pdb_to_df(pdb_file, target_id):
    parser = PDBParser()
    structure = parser.get_structure('', pdb_file)

    df_list = []
    for model in structure:
        for chain in model:
            chain_data = []
            for residue in chain:
                if residue.get_resname() in ['A', 'U', 'G', 'C']:
                    if 'C1\'' in residue:
                        atom = residue['C1\'']
                        x, y, z = atom.get_coord()
                        resid = residue.get_id()[1]
                        chain_data.append(dict(
                            ID=target_id + '_' + str(resid),
                            resname=residue.get_resname(),
                            resid=resid,
                            x_1=x,
                            y_1=y,
                            z_1=z
                        ))
            if chain_data:
                df_list.append(pd.DataFrame(chain_data))
    return df_list

# 📌 좌표 → PDB 라인 생성
def write_target_line(atom_name, atom_serial, residue_name, chain_id,
                      residue_num, x_coord, y_coord, z_coord,
                      occupancy=1.0, b_factor=0.0, atom_type='C'):
    return (
        f'ATOM  {atom_serial:>5d}  {atom_name:<5s} {residue_name:<3s} '
        f'{residue_num:>3d}    {x_coord:>8.3f}{y_coord:>8.3f}{z_coord:>8.3f}'
        f'{occupancy:>6.2f}{b_factor:>6.2f}           {atom_type}\n'
    )

# 📌 DataFrame → PDB 저장
def write_xyz_to_pdb(df, pdb_file, xyz_id=1):
    resolved_cnt = 0
    with open(pdb_file, 'w') as f:
        for _, row in df.iterrows():
            x, y, z = row[f'x_{xyz_id}'], row[f'y_{xyz_id}'], row[f'z_{xyz_id}']
            if x > -1e17 and y > -1e17 and z > -1e17:
                resolved_cnt += 1
                f.write(write_target_line(
                    atom_name="C1'",
                    atom_serial=int(row['resid']),
                    residue_name=row['resname'],
                    chain_id='0',
                    residue_num=int(row['resid']),
                    x_coord=x, y_coord=y, z_coord=z,
                    atom_type='C'
                ))
    return resolved_cnt

# 📌 USalign → TM-score 추출
def parse_usalign_for_tm_score(output):
    matches = re.findall(r'TM-score=\s+([\d.]+)', output)
    if len(matches) < 2:
        raise ValueError("TM-score 추출 실패")
    return float(matches[1])

# 📌 USalign → 회전행렬 추출
def parse_usalign_for_transform(output):
    lines = output.splitlines()
    matrix_lines = []
    in_matrix = False
    for line in lines:
        if "The rotation matrix to rotate Structure_1 to Structure_2" in line:
            in_matrix = True
        elif in_matrix and re.match(r'^\d+\s+[-\d.]+\s+[-\d.]+\s+[-\d.]+\s+[-\d.]+$', line):
            matrix_lines.append(line)
        elif in_matrix and not line.strip():
            break
    matrix = [list(map(float, l.split()[1:])) for l in matrix_lines]
    return np.array(matrix)

# 📌 평가 전체 실행
def call_usalign(predict_df, truth_df, usalign_bin=USALIGN, verbose=True):
    pred_pdb, gt_pdb = '~pred.pdb', '~truth.pdb'
    write_xyz_to_pdb(predict_df, pred_pdb)
    write_xyz_to_pdb(truth_df, gt_pdb)
    cmd = f'{usalign_bin} {pred_pdb} {gt_pdb} -atom " C1\'" -m -'
    output = os.popen(cmd).read()
    if verbose:
        print(output)
    return parse_usalign_for_tm_score(output), parse_usalign_for_transform(output)

print('✅ RNA 구조 평가용 HELPER 로딩 완료!')


✅ RNA 구조 평가용 HELPER 로딩 완료!


In [ ]:
if MODEL_TYPE=='protenix':


    from runner.batch_inference import get_default_runner
    from runner.inference import update_inference_configs, InferenceRunner

    from protenix.data.infer_data_pipeline import InferenceDataset

    np.random.seed(0)
    torch.random.manual_seed(0)
    torch.cuda.manual_seed_all(0)

    class DictDataset:
        def __init__(
            self,
            seq_list: list,
            dump_dir: str,
            id_list: list = None,
            use_msa: bool = False,
        ) -> None:
            assert isinstance(seq_list, list), "seq_list는 리스트여야 합니다"
            if id_list is not None:
                assert len(seq_list) == len(id_list), "seq_list와 id_list 길이 다름"

            self.dump_dir = dump_dir
            self.use_msa = use_msa

            if id_list is None:
                self.inputs = [{
                    "sequences": [{
                        "rnaSequence": {
                            "sequence": seq,
                            "count": 1
                        }
                    }],
                    "name": f"query_{idx}"
                } for idx, seq in enumerate(seq_list)]
            else:
                self.inputs = [{
                    "sequences": [{
                        "rnaSequence": {
                            "sequence": seq,
                            "count": 1
                        }
                    }],
                    "name": id_
                } for id_, seq in zip(id_list, seq_list)]

        def __len__(self):
            return len(self.inputs)

        def __getitem__(self, index):
            # InferenceDataset이 제공하는 방식과 유사하게 반환
            from protenix.data.infer_data_pipeline import convert_input_to_data
            input_json = self.inputs[index]
            data, atom_array, error_message = convert_input_to_data(input_json, dump_dir=self.dump_dir, use_msa=self.use_msa)
            return data, atom_array, error_message




In [ ]:
import copy
from distutils.util import strtobool  # 문자열 to bool 변환

if MODEL_TYPE == 'protenix':

    # configs import
    from configs.configs_base import configs as configs_base
    from configs.configs_data import data_configs
    from configs.configs_inference import inference_configs
    from protenix.config.config import parse_configs
    from runner.inference import InferenceRunner

    # ✅ config 깊은 복사 → 원본 보존
    base_config = copy.deepcopy(configs_base)

    # ✅ 환경 변수 boolean 처리 개선
    base_config["use_deepspeed_evo_attention"] = bool(strtobool(os.environ.get("USE_DEEPSPEED_EVO_ATTENTION", "false")))

    # ✅ 모델, 샘플링 하이퍼파라미터 수정
    base_config["model"]["N_cycle"] = 10
    base_config["sample_diffusion"]["N_sample"] = 5 if not VALIDATION else 1
    base_config["sample_diffusion"]["N_step"] = 200

    # ✅ 체크포인트 경로 지정
    inference_configs['load_checkpoint_path'] = '/kaggle/input/protenix-checkpoints/model_v0.2.0.pt'

    # ✅ configs 병합 및 파싱
    configs = {**base_config, **{"data": data_configs}, **inference_configs}
    configs = parse_configs(
        configs=configs,
        fill_required_with_null=True,
    )

    # ✅ Inference Runner 준비
    runner = InferenceRunner(configs)

print("✅ 모델 로딩 완료 여부:", runner.model is not None)
print("📂 체크포인트 경로:", configs['load_checkpoint_path'])


train scheduler 16.0
inference scheduler 16.0
Diffusion Module has 16.0
✅ 모델 로딩 완료 여부: True
📂 체크포인트 경로: /kaggle/input/protenix-checkpoints/model_v0.2.0.pt


In [ ]:
import pandas as pd
import numpy as np
import time
import warnings
from tqdm import tqdm

if VALIDATION:
    LABEL_DF = pd.read_csv('/kaggle/input/stanford-rna-3d-folding/train_labels.csv')
    LABEL_DF['target_id'] = LABEL_DF['ID'].apply(lambda x: '_'.join(x.split('_')[:-1]))
    train_df = pd.read_csv('/kaggle/input/stanford-rna-3d-folding/train_sequences.csv')

    # ✅ 여기 추가
    def get_truth_df(target_id):
        truth_df = LABEL_DF[LABEL_DF['target_id'] == target_id]
        return truth_df.reset_index(drop=True)

if MODEL_TYPE == 'protenix' and VALIDATION:
    warnings.filterwarnings("ignore")

    time0 = time.time()

    train_df['protenix_tm_score'] = None

    # ⬇️ 여기가 수정되는 위치
    dataset = DictDataset(
        seq_list=train_df.sequence.tolist(),
        dump_dir='output',
        id_list=train_df.target_id.tolist(),
        use_msa=False
    )

    num_data = len(dataset)

    for i, seq in tqdm(enumerate(train_df.sequence), total=num_data):
        if pd.notnull(train_df.loc[i, 'protenix_tm_score']):
            continue
        if len(seq) > 300:
            continue

        target_id = train_df.loc[i, 'target_id']
        truth_df = get_truth_df(target_id)

        if sum(~np.isnan(truth_df.x_1)) < 3:
            continue

        data, atom_array, data_error_message = dataset[i]
        if data_error_message != '':
            continue

        try:
            new_configs = update_inference_configs(configs, data["N_token"].item())
            runner.update_model_configs(new_configs)

            prediction = runner.predict(data)
            prediction = prediction['coordinate'][:, data['input_feature_dict']['atom_to_tokatom_idx'] == 12]

            # Optional smoothing (TM-score 향상 여지 있음)
            # def smooth_coords(coords, window=3):
            #     coords = coords.squeeze()
            #     smoothed = np.copy(coords)
            #     for j in range(1, len(coords)-1):
            #         smoothed[j] = np.mean(coords[j-1:j+2], axis=0)
            #     return smoothed
            # prediction = smooth_coords(prediction[:1])

            result = parse_output_to_df(prediction[:1], seq, target_id)[0]

            tm_score, transform = call_usalign(result, truth_df, verbose=0)
            train_df.loc[i, 'protenix_tm_score'] = tm_score

        except Exception as e:
            # Optional debug: print(f"⚠️ {target_id} 평가 실패: {e}")
            continue

        if (time.time() - time0) > (12 * 3600 - 360):
            print("⚠️ 실행 시간 한도 도달로 중단")
            break

    # ✅ 결과 저장
    train_df.to_csv('/kaggle/working/tm_scores.csv', index=False)

    # ✅ 통계 출력 및 시각화
    print("📈 평균 TM-score:", train_df['protenix_tm_score'].astype(float).mean())
    train_df['protenix_tm_score'].astype(float).hist(bins=50)


  0%|          | 0/844 [00:00<?, ?it/s]


ImportError: cannot import name 'convert_input_to_data' from 'protenix.data.infer_data_pipeline' (/usr/local/lib/python3.10/dist-packages/protenix/data/infer_data_pipeline.py)

In [ ]:
if MODEL_TYPE=='protenix' and not VALIDATION:
    test_df=pd.read_csv('/kaggle/input/stanford-rna-3d-folding/test_sequences.csv')
    import warnings
    warnings.filterwarnings("ignore")

    dataset = DictDataset(test_df.sequence, dump_dir='output', id_list=test_df.target_id, use_msa=False)
    num_data = len(dataset)
    for i, seq in tqdm(enumerate(test_df.sequence),total=num_data):
        try:
            data, atom_array, data_error_message=dataset[i]
            target_id = data["sample_name"]
            assert target_id==test_df.target_id[i]
            assert data_error_message==''

            new_configs = update_inference_configs(configs, data["N_token"].item())
            runner.update_model_configs(new_configs)
            prediction = runner.predict(data)
            prediction=prediction['coordinate'][:,data['input_feature_dict']['atom_to_tokatom_idx']==12]

            result = parse_output_to_df(prediction, seq, target_id)[0]
        except:
            target_id==test_df.target_id[i]
            print('Failed to predict', target_id)
            result=pd.DataFrame(columns=['ID', 'resname', 'resid',
                                         'x_1', 'y_1', 'z_1',
                                         'x_2', 'y_2', 'z_2',
                                         'x_3', 'y_3', 'z_3',
                                         'x_4', 'y_4', 'z_4',
                                         'x_5', 'y_5', 'z_5'],
                                         data=[[target_id, x, j+1] + [0.0]*15 for j, x in enumerate(seq)])

        result['ID']=result.apply(lambda x: x.ID + '_' + str(x.resid), axis=1)
        result.to_csv('submission.csv', index=False, mode='a', header=(i==0))
        torch.cuda.empty_cache()

    display(pd.read_csv('submission.csv'))